In [1]:
# data organization libraries
import numpy as np
import pandas as pd

# data visualization libraries
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# modeling libraries
from sklearn.linear_model import LinearRegression


In [2]:
fires = pd.read_csv('raw_df.csv')

fires.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,area_cat,temp_bins
0,1,2,8,5,91.0,166.9,752.6,7.1,25.9,41,3.6,0.0,0.00,0-5,>25
1,5,4,8,2,95.1,141.3,605.8,17.7,26.4,34,3.6,0.0,16.40,10-50,>25
2,6,5,8,3,92.1,111.2,654.1,9.6,16.6,47,0.9,0.0,2.29,0-5,15-20
3,6,3,8,4,91.6,138.1,621.7,6.3,18.9,41,3.1,0.0,10.34,10-50,15-20
4,4,5,8,0,90.2,99.6,631.2,6.3,21.4,33,3.1,0.0,0.00,0-5,20-25


In [3]:
len(fires)

2652

In [4]:
# create model that trains on all features


In [5]:
#create naive model
# need to split data into X(features) and y(response)

In [6]:
X_df = fires.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11]]

X_df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain
0,1,2,8,5,91.0,166.9,752.6,7.1,25.9,41,3.6,0.0
1,5,4,8,2,95.1,141.3,605.8,17.7,26.4,34,3.6,0.0
2,6,5,8,3,92.1,111.2,654.1,9.6,16.6,47,0.9,0.0
3,6,3,8,4,91.6,138.1,621.7,6.3,18.9,41,3.1,0.0
4,4,5,8,0,90.2,99.6,631.2,6.3,21.4,33,3.1,0.0


In [7]:
y_df = fires.iloc[:, [12]]

y_df.head()

,area
0,0.00
1,16.40
2,2.29
3,10.34
4,0.00


In [8]:
X = X_df.to_numpy()

X

array([[ 1. ,  2. ,  8. , ..., 41. ,  3.6,  0. ],
       [ 5. ,  4. ,  8. , ..., 34. ,  3.6,  0. ],
       [ 6. ,  5. ,  8. , ..., 47. ,  0.9,  0. ],
       ...,
       [ 5. ,  4. ,  4. , ..., 27. ,  5.8,  0. ],
       [ 6. ,  5. ,  4. , ..., 54. ,  5.8,  0. ],
       [ 6. ,  5. ,  4. , ..., 64. ,  3.1,  0. ]])

In [9]:
y = y_df.to_numpy()

y = y.flatten()

y

array([ 0.  , 16.4 ,  2.29, ...,  0.  , 10.93,  3.35])

In [10]:
# lets do a simple test train split using sklearn
from sklearn.model_selection import train_test_split


In [11]:
# pre split
X.shape, y.shape

((2652, 12), (2652,))

In [12]:
# split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

In [13]:
# post split
X_train.shape, y_train.shape

((2121, 12), (2121,))

In [14]:
# post split
X_test.shape, y_test.shape

((531, 12), (531,))

In [15]:
# normalize, i.e. standard scalar
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(data)

print(scaler.mean_)

print(scaler.transform(data))




print(scaler.transform([[2, 2]]))


In [22]:
# fit train
model = LinearRegression().fit(X_train, y_train)

In [23]:
# predict
y_hat = model.predict(X_test)



In [24]:
# error
from sklearn.metrics import mean_squared_error

error = mean_squared_error(y_test, y_hat)

error

200.66535864299453

In [26]:
score = ((y_test - y_hat)**2).sum()

score

106553.3054394301

In [31]:
# score
score = model.score(X_test, y_test)

score

0.009302190398110644

In [20]:
# plot residuals
#sns.residplot(x=y_hat,y=y_test)
